<a href="https://colab.research.google.com/github/yesequin/Preprocessing-and-data-exploration_GermanBank/blob/main/Preprocessing_and_data_exploration_GermanBank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA IMMERSION WITH PYTHON

In [ ]:
# prompt: import the next modules with their respective alias: pandas, matplotlib, seaborn, drive from google colab, warnings

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import warnings

drive.mount('/content/Drive')
warnings.filterwarnings('ignore') #ignore warnings.



Mounted at /content/Drive


In [ ]:
route = '/content/Drive/MyDrive/DATA SCIENCE/INMERSION DATOS/German credit project/german_credit.csv'
pd.set_option('display.max_columns',None) #make the number of columns to be displayed unlimited. This let me see all the columns available.
global df_bank, results

df_bank = pd.read_csv(route)
df_bank.head()


,default,account_check_status,duration_in_month,credit_history,purpose,credit_amount,savings,present_emp_since,installment_as_income_perc,personal_status_sex,other_debtors,present_res_since,property,age,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker
0,0,< 0 DM,6,critical account/ other credits existing (not ...,domestic appliances,1169,unknown/ no savings account,.. >= 7 years,4,male : single,none,4,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes
1,1,0 <= ... < 200 DM,48,existing credits paid back duly till now,domestic appliances,5951,... < 100 DM,1 <= ... < 4 years,2,female : divorced/separated/married,none,2,real estate,22,none,own,1,skilled employee / official,1,none,yes
2,0,no checking account,12,critical account/ other credits existing (not ...,(vacation - does not exist?),2096,... < 100 DM,4 <= ... < 7 years,2,male : single,none,3,real estate,49,none,own,1,unskilled - resident,2,none,yes
3,0,< 0 DM,42,existing credits paid back duly till now,radio/television,7882,... < 100 DM,4 <= ... < 7 years,2,male : single,guarantor,4,if not A121 : building society savings agreeme...,45,none,for free,1,skilled employee / official,2,none,yes
4,1,< 0 DM,24,delay in paying off in the past,car (new),4870,... < 100 DM,1 <= ... < 4 years,3,male : single,none,4,unknown / no property,53,none,for free,2,skilled employee / official,2,none,yes


In [ ]:
df_bank.info() #we can check if there are non-null data, and what datatype you will work with.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   default                     1000 non-null   int64 
 1   account_check_status        1000 non-null   object
 2   duration_in_month           1000 non-null   int64 
 3   credit_history              1000 non-null   object
 4   purpose                     1000 non-null   object
 5   credit_amount               1000 non-null   int64 
 6   savings                     1000 non-null   object
 7   present_emp_since           1000 non-null   object
 8   installment_as_income_perc  1000 non-null   int64 
 9   personal_status_sex         1000 non-null   object
 10  other_debtors               1000 non-null   object
 11  present_res_since           1000 non-null   int64 
 12  property                    1000 non-null   object
 13  age                         1000 non-null   int64

In [ ]:
# First step is take all 'object' datatype and evalute what categories we have in those columns.
#so we create a for loop to see the column names and the unique categories, to see how we can convert those in numeric values
column = list(df_bank.select_dtypes(include=['object']).columns) #creating a list with all the columns with dtype = object

for col in column:
  category = list(df_bank[col].value_counts().index) #value_counts create a series with the unique category names as an index and the value as the first column. With .index we gather only the index names
  print(f'Name column is: {col}, with the next categories: {category}')
  print('\n')

Name column is: account_check_status, with the next categories: ['no checking account', '< 0 DM', '0 <= ... < 200 DM', '>= 200 DM / salary assignments for at least 1 year']


Name column is: credit_history, with the next categories: ['existing credits paid back duly till now', 'critical account/ other credits existing (not at this bank)', 'delay in paying off in the past', 'all credits at this bank paid back duly', 'no credits taken/ all credits paid back duly']


Name column is: purpose, with the next categories: ['domestic appliances', 'car (new)', 'radio/television', 'car (used)', 'business', '(vacation - does not exist?)', 'education', 'repairs', 'furniture/equipment', 'retraining']


Name column is: savings, with the next categories: ['... < 100 DM', 'unknown/ no savings account', '100 <= ... < 500 DM', '500 <= ... < 1000 DM ', '.. >= 1000 DM ']


Name column is: present_emp_since, with the next categories: ['1 <= ... < 4 years', '.. >= 7 years', '4 <= ... < 7 years', '... < 1 yea

NOTE: The German Credit data.csv file has the information about what numeric value should have each category. THIS DOCUMENT IS A DICTIONARY

In [ ]:
#now we convert those categoric values into numeric values with the df_bank function
def data_processing():
  global df_bank
  df_bank = df_bank.drop_duplicates() if df_bank.duplicated().any() else df_bank #we make sure there is no duplicate data
  df_bank = df_bank.dropna() if df_bank.isnull().values.any() else df_bank #we make sure there is no NULL data

  dic_account_check_status = {'no checking account': 4,
                              '< 0 DM': 1,
                              '0 <= ... < 200 DM': 2,
                              '>= 200 DM / salary assignments for at least 1 year': 3
  }
  dic_credit_history = {'existing credits paid back duly till now': 3,
                        'critical account/ other credits existing (not at this bank)': 5,
                        'delay in paying off in the past': 4,
                        'all credits at this bank paid back duly': 2,
                        'no credits taken/ all credits paid back duly': 1
  }
  dic_purpose = {'domestic appliances': 5,
                 'car (new)': 1,
                 'radio/television': 4,
                 'car (used)': 2,
                 'business': 10,
                 '(vacation - does not exist?)': 8,
                 'education': 7,
                 'repairs': 6,
                 'furniture/equipment': 3,
                 'retraining': 9
  }
  dic_savings = {'... < 100 DM': 5,
                 'unknown/ no savings account': 1,
                 '100 <= ... < 500 DM': 4,
                 '500 <= ... < 1000 DM ':3,
                 '.. >= 1000 DM ': 2
  }
  dic_present_emp_since = {'1 <= ... < 4 years': 3,
                        '.. >= 7 years': 1,
                        '4 <= ... < 7 years': 2,
                        '... < 1 year ': 4,
                        'unemployed':5
  }
  dic_personal_status_sex = {'male : single': 3,
                             'female : divorced/separated/married': 2,
                             'male : married/widowed': 4,
                             'male : divorced/separated': 1
  }
  dic_other_debtors = {'none': 1,
                       'guarantor': 3,
                       'co-applicant': 2
  }
  dic_property = {'if not A121/A122 : car or other, not in attribute 6': 3,
                  'real estate': 1,
                  'if not A121 : building society savings agreement/ life insurance': 2,
                  'unknown / no property': 4
  }
  dic_other_installment_plans = {'none': 3,
                              'bank': 1,
                              'stores': 2
  }
  dic_housing = {'own': 2,
                 'rent': 1,
                 'for free': 3
  }
  dic_job = {'skilled employee / official': 3,
             'unskilled - resident': 2,
             'management/ self-employed/ highly qualified employee/ officer': 4,
             'unemployed/ unskilled - non-resident': 1
  }
  dic_telephone ={'none':0,
                  'yes, registered under the customers name ': 1
  }
  dic_foreign_worker = {'yes':1,
                        'no':0
  }
  #we assign the dictionary value to that specific column with .map
  df_bank['account_check_status'] = df_bank['account_check_status'].map(dic_account_check_status)
  df_bank['credit_history'] = df_bank['credit_history'].map(dic_credit_history)
  df_bank['purpose'] = df_bank['purpose'].map(dic_purpose)
  df_bank['savings'] = df_bank['savings'].map(dic_savings)
  df_bank['present_emp_since'] = df_bank['present_emp_since'].map(dic_present_emp_since)
  df_bank['personal_status_sex'] = df_bank['personal_status_sex'].map(dic_personal_status_sex)
  df_bank['other_debtors'] = df_bank['other_debtors'].map(dic_other_debtors)
  df_bank['property'] = df_bank['property'].map(dic_property)
  df_bank['other_installment_plans'] = df_bank['other_installment_plans'].map(dic_other_installment_plans)
  df_bank['housing'] = df_bank['housing'].map(dic_housing)
  df_bank['job'] = df_bank['job'].map(dic_job)
  df_bank['telephone'] = df_bank['telephone'].map(dic_telephone)
  df_bank['foreign_worker'] = df_bank['foreign_worker'].map(dic_foreign_worker)

data_processing()

In [ ]:
##The less scattered the data is, the better and faster the machine will understand it, so now
###we take all the column with continues variables and convert them into discrete, establishing a range.

discrete_var = ['personal_status_sex','age','duration_in_month','credit_amount','default']
a = df_bank[discrete_var].value_counts()
print(a)

def feature_engineering():
  global df_bank

  dic_genre = {2:1, 1:0, 3:0, 4:0}
  dic_marital_status = {3:1, 5:1, 1:0, 2:0, 4:0}
  df_bank['genre'] = df_bank['personal_status_sex'].map(dic_genre)
  df_bank['marital_status'] = df_bank['personal_status_sex'].map(dic_marital_status)
  df_bank['age_range'] = pd.cut(x=df_bank['age'], #cutting the data into ranges
                                bins=[18, 30, 40, 50, 60, 70, 80], #this will be the range values
                                labels = [1, 2, 3, 4, 5, 6]).astype(int) #this will be the new values: an age from 18 to 30 will be 1, 31 to 40 will be 2...
  df_bank['credit_term_range'] = pd.cut(x=df_bank['duration_in_month'],
                                        bins=[1, 12, 24, 36, 48, 60, 72],
                                        labels = [1, 2, 3, 4, 5, 6]).astype(int)
  df_bank['credit_amount_range'] = pd.cut(x=df_bank['credit_amount'],
                                          bins=[1, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000,
                                                14000, 15000, 16000, 17000, 18000, 19000, 20000],
                                          labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]).astype(int)
  df_bank['credit_term_range'] = pd.cut(x=df_bank['duration_in_month'],
                                        bins=[1, 12, 24, 36, 48, 60, 72],
                                        labels = [1, 2, 3, 4, 5, 6]).astype(int)

  df_bank = df_bank.drop(columns=['personal_status_sex','age', 'duration_in_month','credit_amount']) #deleting the replaced columns

feature_engineering()
df_bank.head(10)



In [ ]:
df_bank.head(20)

,default,account_check_status,credit_history,purpose,savings,present_emp_since,installment_as_income_perc,other_debtors,present_res_since,property,other_installment_plans,housing,credits_this_bank,job,people_under_maintenance,telephone,foreign_worker,genre,marital_status,age_range,credit_term_range,credit_amount_range
0,0,1,5,5,1,1,4,1,4,1,3,2,2,3,1,1,1,0,1,5,1,2
1,1,2,3,5,5,3,2,1,2,1,3,2,1,3,1,0,1,1,0,1,4,6
2,0,4,5,8,5,2,2,1,3,1,3,2,1,2,2,0,1,0,1,3,1,3
3,0,1,3,4,5,2,2,3,4,2,3,3,1,3,2,0,1,0,1,3,4,8
4,1,1,4,1,5,3,3,1,4,4,3,3,2,3,2,0,1,0,1,4,2,5
5,0,4,3,8,1,3,2,1,4,4,3,3,1,2,2,1,1,0,1,2,3,10
6,0,4,3,4,3,1,3,1,4,2,3,2,1,3,1,0,1,0,1,4,2,3
7,0,2,3,2,5,3,2,1,2,3,3,1,1,4,1,1,1,0,1,2,3,7
8,0,4,3,5,2,2,2,1,4,1,3,2,1,2,1,0,1,0,0,5,1,4
9,1,2,5,1,5,5,4,1,2,3,3,2,2,4,1,0,1,0,0,1,3,6


In [ ]:
# variables = ['genre','age_range']

# fig, ax = plt.subplots(nrows=2, ncols=1,figsize= (15,25))
# fig.subplots_adjust(hspace=1)

# for i,col in enumerate(variables):
#   sns.countplot(x=col, data=df_bank, ax=ax[i])
#   ax[i].set_title(col)
#   ax[i].set_xticklabels(ax[i].get_xticklabels(),rotation=30)

def graphs():
  global df_banco
  histograms = ['genre','marital_status','credit_term_range','age_range','default']
  histograms_list = list(enumerate(histograms))
  plt.figure(figsize = (30,20))
  plt.title('Histograms')
  for i in histograms_list:
    plt.subplot(3, 2, i[0]+1) #3 columns, 2 rows
    sns.countplot(x = i[1], data = df_bank)
    plt.xlabel(i[1], fontsize=20)
    plt.ylabel('Total', fontsize=20)
